In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LogisticRegression
import sklearn
import pandas_datareader.data as web
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

C:\Users\Bright\anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
def create_dataset(stock_symbol,start_date ,end_date, lags=5):
    
    df = web.DataReader(stock_symbol, "yahoo", start_date, end_date)
    
    tslag=pd.DataFrame(index=df.index)
    tslag['Today']=df['Adj Close']
    tslag['Volume']=df['Volume']
    
    for i in range(0, lags):
        tslag["Lag%s" % str(i+1)] = df['Adj Close'].shift(i+1)
        
    
    dfret = pd.DataFrame(index=tslag.index)
    dfret['Volume'] = tslag['Volume']
    dfret['Today'] = tslag['Today'].pct_change()*100.0
    
    for i in range(0, lags):
        dfret["Lag%s" % str(i+1)] = tslag['Lag%s' % str(i+1)].pct_change()*100.0
    
    dfret['Direction'] = np.sign(dfret['Today'])
    
    dfret.drop(dfret.index[:5], inplace=True)
    
    return dfret

In [3]:
data = create_dataset("AAPL", datetime(2012,1,1), datetime(2017,5,31), lags=5)

X=data[["Lag1", "Lag2", "Lag3", "Lag4"]]
Y=data["Direction"]


start_test = datetime(2017,1,1)

X_train = X[X.index < start_test]
X_test = X[X.index >= start_test]
Y_train = Y[Y.index < start_test]
Y_test = Y[Y.index >= start_test]

model = KNeighborsClassifier(300)

model.fit(X_train, Y_train)

pred = model.predict(X_test)


print("Accuracy of kNN model: %0.3f" % model.score(X_test, Y_test))
print("Confusion matrix: \n%s" % confusion_matrix(pred, Y_test))

C:\Users\Bright\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in sign
  result = getattr(ufunc, method)(*inputs, **kwargs)


Accuracy of kNN model: 0.612
Confusion matrix: 
[[17 13]
 [27 46]]
